In [2]:
import os
import logging
from pyspark.sql import SparkSession

#os.environ["PYSPARK_PYTHON"] = '/opt/.venv/bin/python'
#os.environ["SPARK_HOME"] = '/opt/spark'

def init_spark():
    """
    Initializes SparkSession and returns SparkSession and SparkContext objects
    
    Returns:
        SparkSession, SparkContext: objects of SparkSession and SparkContext respectively
    """
    logging.info("Initializing Spark session")
    spark = SparkSession.builder\
                        .appName("query")\
                        .master("spark://spark-master:7077")\
                        .config("spark.jars", "/opt/workspace/spark-apps/packages/postgresql-42.2.22.jar,/opt/workspace/spark-apps/packages/spark-avro_2.12-3.0.2.jar")\
                        .config("spark.executor.memory", "512m")\
                        .getOrCreate()
    logging.info("Spark session initialized successfully")
    return spark

spark = init_spark()

23/02/09 09:30:51 INFO CoarseGrainedSchedulerBackend$DriverEndpoint: Registered executor NettyRpcEndpointRef(spark-client://Executor) (172.23.0.6:55148) with ID 0
23/02/09 09:30:51 INFO CoarseGrainedSchedulerBackend$DriverEndpoint: Registered executor NettyRpcEndpointRef(spark-client://Executor) (172.23.0.7:60686) with ID 1
23/02/09 09:30:52 INFO BlockManagerMasterEndpoint: Registering block manager 172.23.0.7:37293 with 127.2 MiB RAM, BlockManagerId(1, 172.23.0.7, 37293, None)
23/02/09 09:30:52 INFO BlockManagerMasterEndpoint: Registering block manager 172.23.0.6:34101 with 127.2 MiB RAM, BlockManagerId(0, 172.23.0.6, 34101, None)


In [2]:
df = spark.read.load('/opt/workspace/data/historic/departments/*.csv', format="csv", inferSchema="true", sep="\t")
df.show()

+--------------------+
|                 _c0|
+--------------------+
|1,Product Management|
|             2,Sales|
|3,Research and De...|
|4,Business Develo...|
|       5,Engineering|
|   6,Human Resources|
|          7,Services|
|           8,Support|
|         9,Marketing|
|         10,Training|
|            11,Legal|
|       12,Accounting|
+--------------------+



In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
            StructField("id", IntegerType(), True),
            StructField("department", StringType(), True)
          ])

df = spark.read.load('/opt/workspace/data/historic/departments/*.csv', format="csv", schema=schema, sep=",")
df.show()

+---+--------------------+
| id|          department|
+---+--------------------+
|  1|  Product Management|
|  2|               Sales|
|  3|Research and Deve...|
|  4|Business Development|
|  5|         Engineering|
|  6|     Human Resources|
|  7|            Services|
|  8|             Support|
|  9|           Marketing|
| 10|            Training|
| 11|               Legal|
| 12|          Accounting|
+---+--------------------+



In [3]:
df.drop('id')

DataFrame[department: string]

In [4]:
df.show()

+---+--------------------+
| id|          department|
+---+--------------------+
|  1|  Product Management|
|  2|               Sales|
|  3|Research and Deve...|
|  4|Business Development|
|  5|         Engineering|
|  6|     Human Resources|
|  7|            Services|
|  8|             Support|
|  9|           Marketing|
| 10|            Training|
| 11|               Legal|
| 12|          Accounting|
+---+--------------------+



In [19]:
url = "jdbc:postgresql://postgres_database:5432/globant_challenge_db"
properties = {
    "user": "globant_super_admin",
    "password": "pass1234",
    "driver": "org.postgresql.Driver"
}

df = df.drop('id')

df.write \
    .format("jdbc") \
    .option("url", url) \
    .option("dbtable", "dbo.departments") \
    .option("user", "globant_super_admin") \
    .option("password", "pass1234") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()

In [11]:
df.write.format('jdbc').options('url', url).options('table', 'dbo.department').options(mode+'append')

TypeError: options() takes 1 positional argument but 3 were given

In [3]:
url = "jdbc:postgresql://postgres_database:5432/globant_challenge_db"
properties = {
    "user": "globant_super_admin",
    "password": "pass1234",
    "driver": "org.postgresql.Driver"
}
df = spark.read \
                .format("jdbc") \
                .option("url", url) \
                .option("driver", properties["driver"]) \
                .option("dbtable", f"dbo.departments") \
                .option("user", properties["user"]) \
                .option("password", properties["password"]) \
                .load()

In [4]:
df.show()

23/02/09 09:31:14 INFO CodeGenerator: Code generated in 828.841606 ms
23/02/09 09:31:14 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
23/02/09 09:31:14 INFO DAGScheduler: Got job 0 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
23/02/09 09:31:14 INFO DAGScheduler: Final stage: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0)
23/02/09 09:31:14 INFO DAGScheduler: Parents of final stage: List()
23/02/09 09:31:14 INFO DAGScheduler: Missing parents: List()
23/02/09 09:31:14 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[2] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
23/02/09 09:31:15 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 9.2 KiB, free 434.4 MiB)
23/02/09 09:31:15 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 4.8 KiB, free 434.4 MiB)
23/02/09 09:31:15 INFO BlockManagerInfo: Added broadcast_0_

+---+--------------------+
| id|          department|
+---+--------------------+
|  1|  Product Management|
|  2|               Sales|
|  3|Research and Deve...|
|  4|Business Development|
|  5|         Engineering|
|  6|     Human Resources|
|  7|            Services|
|  8|             Support|
|  9|           Marketing|
| 10|            Training|
| 11|               Legal|
| 12|          Accounting|
+---+--------------------+



23/02/09 09:31:19 INFO TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 3361 ms on 172.23.0.6 (executor 0) (1/1)
23/02/09 09:31:19 INFO TaskSchedulerImpl: Removed TaskSet 0.0, whose tasks have all completed, from pool 
23/02/09 09:31:19 INFO DAGScheduler: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0) finished in 4.057 s
23/02/09 09:31:19 INFO DAGScheduler: Job 0 is finished. Cancelling potential speculative or zombie tasks for this job
23/02/09 09:31:19 INFO TaskSchedulerImpl: Killing all running tasks in stage 0: Stage finished
23/02/09 09:31:19 INFO DAGScheduler: Job 0 finished: showString at NativeMethodAccessorImpl.java:0, took 4.131217 s
23/02/09 09:31:19 INFO CodeGenerator: Code generated in 21.908301 ms            


In [5]:
df.write.format("avro").save("/opt/workspace/data/backups/departments/departments.avro")

AnalysisException: Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of "Apache Avro Data Source Guide".;

23/02/09 09:36:23 INFO BlockManagerInfo: Removed broadcast_0_piece0 on f7a220ef6852:35751 in memory (size: 4.8 KiB, free: 434.4 MiB)
23/02/09 09:36:23 INFO BlockManagerInfo: Removed broadcast_0_piece0 on 172.23.0.6:34101 in memory (size: 4.8 KiB, free: 127.2 MiB)
